# Import data and perform classifications (Example 1)

In this notebook we:
* (1) Import pose-estimation data to a SimBA project (with optional data cleaning smoothing / interpolation), and
* (2) Calculate features from our pose-estimation data, and
* (3) Run classification on our data, and finally
* (4) Compute some optional aggregate movement statistics from our pose-estimation data.

> Note: Check the comments below for user-settings and additional information. And reach out to us on [Gitter](https://app.gitter.im/#/room/#SimBA-Resource_community:gitter.im) or [GitHub](https://github.com/sgoldenlab) if you hit any blockers and we will work to resolve it.

In [1]:
import simba # version >= 1.59.3
from simba.pose_importers.dlc_importer_csv import import_multiple_dlc_tracking_csv_file
from simba.outlier_tools.skip_outlier_correction import OutlierCorrectionSkipper
from simba.utils.cli.cli_tools import feature_extraction_runner, set_video_parameters
from simba.model.inference_batch import InferenceBatch
from simba.data_processors.agg_clf_calculator import AggregateClfCalculator
from simba.data_processors.movement_calculator import MovementCalculator

/Users/simon/opt/anaconda3/envs/Python_3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [14]:
!pip show simba-uw-tf-dev

Name: Simba-UW-tf-dev
Version: 1.94.9
Summary: Toolkit for computer classification of behaviors in experimental animals
Home-page: https://github.com/sgoldenlab/simba
Author: Simon Nilsson, Jia Jie Choong, Sophia Hwang
Author-email: sronilsson@gmail.com
License: GNU Lesser General Public License v3 (LGPLv3)
Location: /Users/simon/opt/anaconda3/envs/simba/lib/python3.6/site-packages
Requires: pandas, dash-core-components, scikit-image, scipy, matplotlib, eli5, kaleido, numpy, dash-html-components, pyyaml, xlrd, shapely, numba, dash, shap, cefpython3, seaborn, ffmpeg-python, opencv-python, joblib, imutils, dash-colorscales, tqdm, xgboost, plotly, psutil, dash-color-picker, tabulate, numexpr, trafaret, imbalanced-learn, Pillow, yellowbrick, scikit-learn, dtreeviz, pyarrow, imgaug, graphviz, geos, h5py, tables, statsmodels
Required-by: 


In [15]:
# DEFINITIONS: HERE WE (i) DEFINE THE PATHS TO OUR DATA / SIMBA PROJECT, (ii) HOW WE SHOULD INTERPOLATE / SMOOTH OUR NEW 
# DATA, AND (III) THE ATTRIBUTES OF OUR NEW VIDEOS (FPS ETC.)

## Define the path to our SimBA project config ini
CONFIG_PATH = '/Users/simon/Desktop/envs/simba/troubleshooting/levi/project_folder/project_config.ini'

## Define the path to the directory holding our new DLC CSV pose-estimation data 
DATA_DIR = '/Users/simon/Desktop/envs/simba/troubleshooting/levi/import_data'

## Define if / how you want to interpolate missing pose-estimation data,
## and if/how you want to smooth the new pose estimation data: here we do neither.
INTERPOLATION_SETTING = 'Body-parts: Nearest' # OPTIONS: 'None', Animal(s): Nearest', 'Animal(s): Linear', 'Animal(s): Quadratic','Body-parts: Nearest', 'Body-parts: Linear', 'Body-parts: Quadratic'
SMOOTHING_SETTING = None # OPTIONS: 'Gaussian', 'Savitzky Golay'
SMOOTHING_TIME = None # TIME IN MILLISECOND

## Define the fps and the pixels per millimeter of the incoming data: has to be the same for all new videos.
## if you have varying fps / px per millimeter / resolutions, then use gui (2023/05)
FPS = 15
PX_PER_MM = 4.6
RESOLUTION = (600, 400) # WIDTH X HEIGHT

In [17]:
# WE RUN THE DATA IMPORTER FOR OUR DIRECTORY OF FILES
## This imports your DLC files in the ``DATA_DIR`` according to the smoothing / interpolation settings defined above
import_multiple_dlc_tracking_csv_file(config_path=CONFIG_PATH,
                                     interpolation_setting=INTERPOLATION_SETTING,
                                     smoothing_setting=SMOOTHING_SETTING,
                                     smoothing_time=SMOOTHING_TIME,
                                     data_dir=DATA_DIR)

Importing 6676 to SimBA project...
Pose-estimation data for video 6676 imported to SimBA project (elapsed time: 1.0319s)...
Video 6676 interpolated (elapsed time 1.7966) ...
SIMBA COMPLETE: 1 data file(s) interpolated) (elapsed time: 1.8036s) 	complete
SIMBA COMPLETE: Imported 1 pose estimation file(s) (elapsed time: 2.8514s) 	complete


In [5]:
# RUN OUTLIER CORRECTION. HERE, WE SPECIFY THAT WE WANT TO SKIP OUTLIER CORRECTION.  
OutlierCorrectionSkipper(config_path=CONFIG_PATH).run()

Processing 3 file(s)...
Skipped outlier correction for video Aqu_FFJ_Cre_721 (elapsed time 0.1739s)...
Skipped outlier correction for video Aqu_FFJ_Cre_723 (elapsed time 0.1518s)...
Skipped outlier correction for video Aqu_FFJ_Cre_722 (elapsed time 0.1434s)...
SIMBA COMPLETE: Skipped outlier correction for 3 files (elapsed time: 0.4862s) 	complete


In [6]:
# SET THE VIDEO PARAMETERS FOR THE NEW VIDEOS. 
## This command inserts your new videos in the ``project_folder/logs/video_info.csv`` according to your video attributes specified above.
set_video_parameters(config_path=CONFIG_PATH, px_per_mm=PX_PER_MM, fps=FPS, resolution=RESOLUTION)

SIMBA COMPLETE: Video parameters set (elapsed time: 0.0146s) 	complete


In [7]:
# COMPUTE FEATURES
# Here we run feature extraction for all the files found in the ``project_folder/csv/outlier_corrected_movement_location`` directory.
feature_extraction_runner(config_path=CONFIG_PATH)

Pose-estimation body part setting for feature extraction: 1 animals 1 body-parts
Extracting features from 3 file(s)...
Processing Aqu_FFJ_Cre_721 (5401 frames)...
Calculating hull variables...
Calculating rolling windows: medians, medians, and sums...
Calculating angles...
Calculating deviations...
Calculating percentile ranks...
Calculating path tortuosities...
Calculating pose probability scores...
Feature extraction complete for Aqu_FFJ_Cre_721 (1/3 (elapsed time: 2.5067s)...
Processing Aqu_FFJ_Cre_723 (5401 frames)...
Calculating hull variables...
Calculating rolling windows: medians, medians, and sums...
Calculating angles...
Calculating deviations...
Calculating percentile ranks...
Calculating path tortuosities...
Calculating pose probability scores...
Feature extraction complete for Aqu_FFJ_Cre_723 (2/3 (elapsed time: 1.5574s)...
Processing Aqu_FFJ_Cre_722 (5401 frames)...
Calculating hull variables...
Calculating rolling windows: medians, medians, and sums...
Calculating angles

In [8]:
# RUN THE CLASSIFIERS
# > NOTE: the minimum bout length, classifier paths, and classification threshold are specified in your project_config.ini.
# if you want to change these parameters, then change them in the project_config.ini prior to running the batch inference command
InferenceBatch(config_path=CONFIG_PATH).run()

Analyzing 3 file(s) with 3 classifier(s)
Analyzing video Aqu_FFJ_Cre_721...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 2000 out of 2000 | elapsed:    0.5s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 2000 

Predictions created for Aqu_FFJ_Cre_721 ...
Analyzing video Aqu_FFJ_Cre_723...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 2000 out of 2000 | elapsed:    0.5s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 2000 

Predictions created for Aqu_FFJ_Cre_723 ...
Analyzing video Aqu_FFJ_Cre_722...


[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 2000 out of 2000 | elapsed:    0.4s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 2000 

Predictions created for Aqu_FFJ_Cre_722 ...
SIMBA COMPLETE: Machine predictions complete. Files saved in project_folder/csv/machine_results directory (elapsed time: 8.3499s) 	complete


In [9]:
# CLASSIFIER DESCRIPTIVE STATISTICS

## Specify which aggregate statistics and which classifiers
DATA_MEASURES = ['Bout count',
                  'Total event duration (s)',
                  'Mean event bout duration (s)',
                  'Median event bout duration (s)',
                  'First event occurrence (s)',
                  'Mean event bout interval duration (s)',
                  'Median event bout interval duration (s)']
CLASSIFIERS = ['freeze', 'groom', 'escape']

## RUN THE CLASSIFIER AGGREGATE STATISTIC CALCULATOR AND SAVE THE RESULTS TO DISK
agg_clf_results = AggregateClfCalculator(config_path=CONFIG_PATH, data_measures=DATA_MEASURES, classifiers=CLASSIFIERS)
agg_clf_results.run()
agg_clf_results.save()

Analyzing 3 file(s) for 3 classifiers...
Analyzing video Aqu_FFJ_Cre_721...
Analyzing video Aqu_FFJ_Cre_723...
Analyzing video Aqu_FFJ_Cre_722...
SIMBA COMPLETE: Data log saved at /Users/simon/Desktop/envs/troubleshooting/notebook_example/project_folder/logs/data_summary_20230512135845.csv (elapsed time: 0.4487s) 	complete


In [10]:
# MOVEMENT DESCRIPTIVE STATISTICS

## SPECIFY WHICH BODY-PARTS AND WHICH POSE-ESTIMATION CONFIDENCE THRESHOLD
MOVEMENT_BODY_PARTS = ['Animal_1 CENTER OF GRAVITY', 'Animal_1 CENTER OF GRAVITY']
MOVEMENT_THRESHOLD = 0.00

## RUN THE MOVEMENT CALCULATOR AND SAVE THE RESULTS TO DISK
movement_results = MovementCalculator(config_path=CONFIG_PATH, body_parts=MOVEMENT_BODY_PARTS, threshold=MOVEMENT_THRESHOLD)
movement_results.run()
movement_results.save()

Processing 3 video(s)...
Analysing Aqu_FFJ_Cre_721...
Analysing Aqu_FFJ_Cre_723...
Analysing Aqu_FFJ_Cre_722...
SIMBA COMPLETE: Movement log saved in /Users/simon/Desktop/envs/troubleshooting/notebook_example/project_folder/logs/Movement_log_20230512135848.csv (elapsed time: 2.2075s) 	complete
